In [11]:
import pandas as pd
import numpy as np
import json

In [13]:
# this should probably be on a seperate cell so I don't constantly reload the dataframes
fDefPath = 'reviews/yelpReviews/yelp_academic_dataset_'
# constants so I don't have to keep changing names
BS = 'business'
CH = 'checkin'
TI = 'tip'
RW = 'review'
US = 'user'

# constants for the file path
bspath = f'{fDefPath}{BS}.json'
chpath = f'{fDefPath}{CH}.json'
tipath = f'{fDefPath}{TI}.json'
rwpath = f'{fDefPath}{RW}.json'
uspath = f'{fDefPath}{US}.json'

# load files in (right now I'm skipping the reviews part cause it's big and my work laptop doens't have the space)
fYelpNames = [bspath, chpath, tipath]


In [3]:
fdf = {}

# load the datasets
#CHANGE THIS, LETS NOT LOAD THIS THINGS INTO A DICTIONARY
for name in fYelpNames:
    fPath = f'{fDefPath}{name}.json'
    fdf[name] = pd.read_json(fPath, lines=True)

for name in fYelpNames:
    print(fdf[name].columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
Index(['business_id', 'date'], dtype='object')
Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')


In [4]:
bsdf = fdf[BS].dropna(subset=["categories"]).copy()
bsdf['categories'] = bsdf['categories'].str.lower()
bsdf.loc[:,"categoryList"] = bsdf.loc[:,"categories"].str.split(", ")

# Flatten and count unique categories
allCategories = bsdf["categoryList"].explode().str.lower()
categoryCounts = allCategories.value_counts()

# Preview common categories
print(categoryCounts.head(50))

categoryList
restaurants                  52268
food                         27781
shopping                     24395
home services                14356
beauty & spas                14292
nightlife                    12281
health & medical             11890
local services               11198
bars                         11065
automotive                   10773
event planning & services     9895
sandwiches                    8366
american (traditional)        8139
active life                   7687
pizza                         7093
coffee & tea                  6703
fast food                     6472
breakfast & brunch            6239
american (new)                6097
hotels & travel               5857
home & garden                 5799
fashion                       5739
burgers                       5636
arts & entertainment          5434
auto repair                   5433
hair salons                   5046
nail salons                   4621
mexican                       4600
italian

In [9]:
# finding what american new means
#bsdf[bsdf["categories"].str.contains(r"american \(new\)", na=False)].name.head(50)


In [6]:
# create a manual dictionary for now
categoryMap = {
    # removed food, too broad, was catching grocery stores
    'restaurants': {'restaurants', 'sandwiches', 'american (new)', 'american (traditional)', 'pizza', 'bakeries', 'desserts', 'cafes', 'breakfast & brunch'},
    'retail': {'shopping', 'home & garden', 'fashion', 'real estate', 'grocery', 'department stores', 'electronics', 'toys'},
    'beauty': {'beauty & spas', 'hair salons', 'nail salons', 'massage', 'skin care', 'makeup artists'},
    'auto': {'automotive', 'auto repair'},
    'services': {'home services', 'local services', 'event planning & services'},
    'fitness': {'active life', 'fitness & instruction', 'gyms', 'yoga', 'martial arts'},
    'health': {'health & medical', 'doctors', 'dentists'},
    'pets': {'veterinarians', 'pet services', 'pets'},
    'nightlife': {'nightlife', 'bars', 'lounges', 'dance clubs', 'karaoke'},
    'travel': {'hotels & travel', 'hotels', 'tours'},
    'religious': {'religious organizations', 'synagogues'},
    'entertainment': {'festivals', 'arts & entertainment', 'television', 'mass media', 'museums', 'music venues', 'cinema', 'theaters'},
    'finance': {'banks & credit unions', 'financial services'},
    'legal': {'dui law', 'lawyers'},
    'government': {'public services & government', 'libraries'},
    'education': {'education'},
    'professional services': {'employment agencies', 'professional services'}
}


In [7]:
def matchBroadCategory(catList):
    for broadCat, keywords in categoryMap.items():
        if any(cat in keywords for cat in catList):
            return broadCat
    return "other"

bsdf["broadCategory"] = bsdf["categoryList"].apply(matchBroadCategory)


In [8]:
fileName = 'categorized.csv'
bsdf[['name','categoryList','broadCategory']].to_csv(fileName)

In [17]:
# Reviews to looks at
sample_size = 10000
reviews = []
with open(rwpath, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= sample_size:
            break
        reviews.append(json.loads(line))

rdf = pd.DataFrame(reviews)
rdf.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')